In [1]:
!pip install py_vncorenlp
!pip install transformers
!pip install evaluate
!pip install accuracy
!pip install accelerate -U

In [2]:
import py_vncorenlp
!mkdir -p /kaggle/working/vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.2.jar /kaggle/working/vncorenlp/ 
!mv vi-vocab /kaggle/working/vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr /kaggle/working/vncorenlp/models/wordsegmenter/

--2023-10-27 06:50:40--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412703 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.2.jar’

VnCoreNLP-1.2.jar   100%[===================>]  26.14M  --.-KB/s    in 0.1s    

2023-10-27 06:50:40 (255 MB/s) - ‘VnCoreNLP-1.2.jar’ saved [27412703/27412703]

--2023-10-27 06:50:41--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

In [3]:
!mkdir -p /kaggle/output
!mv /kaggle/output /kaggle/working/

In [4]:
!sudo apt-get update
!sudo apt-get install openjdk-8-jdk -y

Hit:1 https://packages.cloud.google.com/apt cloud-sdk InRelease
Hit:2 https://packages.cloud.google.com/apt google-fast-socket InRelease       
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [1299 B]    
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: https://packages.cloud.google.com/apt/dists/google-fast-socket/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: http://packages.cloud.google.com/apt/dists/gcsfuse-focal/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) 

In [5]:
segmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"],save_dir=r"/kaggle/working/vncorenlp")

2023-10-27 06:50:52 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [6]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, AutoModel
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
import ast
import random
import torch.optim as optim

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [7]:
import evaluate

f1 = evaluate.load("f1")

In [8]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

In [19]:
import pandas as pd
df = pd.read_csv('/kaggle/input/combined-bio/combined_BIO.csv', index_col = 0)
# df = df.drop_duplicates(subset=['content'])
df

,content,NER,label
1630,"Tương tự , thì cả 2 dòng điện thoại này vẫn gi...","[0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 7, 8, 5, 6, ...",2
1545,cả hai máy không giống như galaxy m32 5g do T...,"[1, 2, 2, 7, 8, 8, 3, 4, 4, 0, 0, 0, 0, 0]",1
3290,galaxy a 2021 cũng có tấm nền cao cấp lớn nhất...,"[1, 2, 2, 0, 0, 5, 6, 6, 6, 7, 8, 8, 8, 8, 8, ...",5
1584,Do sức mạnh đến từ bộ vi xử lý tương tự nhau n...,"[0, 0, 0, 0, 0, 5, 6, 6, 6, 7, 8, 8, 0, 7, 8, ...",1
1892,selfie được tăng số megapixel trên camera self...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, ...",2
...,...,...,...
2895,các mẫu điện thoại mới ra mắt của nhà samsung ...,"[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, ...",4
2763,con chip này đang là một trong những smartphon...,"[1, 2, 2, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 0]",4
905,Có nghĩa là bạn sẽ trải nghiệm giao diện và ph...,"[0, 0, 0, 0, 0, 5, 6, 6, 6, 0, 5, 6, 7, 8, 0, ...",2
3980,phối cạnh 3x có một màn hình chính nhỏ hơn m...,"[1, 2, 2, 0, 0, 5, 6, 6, 7, 8, 8, 8, 0, 0, 7, ...",7


In [20]:
df['label'].value_counts()

2    601
1    601
5    601
7    601
3    601
8    601
6    601
4    600
Name: label, dtype: int64

In [21]:
df_dct = df[["content","label"]]

In [22]:
id2label = {0:"O",1:"DIF",2:"EQL",3:"SUP+",4:"SUP-",5:"SUP",6:"COM+",7:"COM-",8:"COM"}
label2id = {"O":0,"DIF":1,"EQL":2,"SUP+":3,"SUP-":4,"SUP":5,"COM+":6,"COM-":7,"COM":8}

In [23]:
phobert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", num_labels = 9, id2label = id2label, label2id = label2id).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of the model checkpoint at vinai/phobert-base-v2 were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bi

In [24]:
df_dct["content"] = df_dct["content"].map(lambda x: segmenter.word_segment(x)[0])

/tmp/ipykernel_1999/641992484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dct["content"] = df_dct["content"].map(lambda x: segmenter.word_segment(x)[0])


In [25]:
# train_text, valid_text, train_label, valid_label = train_test_split(df_dct['content'], df_dct['label'], test_size=0.05)

In [26]:
# train_encoding = tokenizer(train_text.to_list(), padding =True)
# valid_encoding = tokenizer(valid_text.to_list(), padding =True)

In [27]:
class DatasetSequenceVLSP(torch.utils.data.Dataset):
  def __init__(self, encoding, labels):
        self.encoding = encoding
        self.labels = labels

  def __len__(self):
        return len(self.encoding['input_ids'])

  def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encoding.items()}
        item["labels"] = torch.tensor([self.labels.iloc[idx]])
        return item

In [28]:
# train_dataset = DatasetSequenceVLSP(train_encoding,train_label)
# valid_dataset = DatasetSequenceVLSP(valid_encoding,valid_label)
skf = StratifiedKFold(n_splits=3, shuffle=True)

train_datasets, valid_datasets = [], []
for train, test in skf.split(df_dct['content'], df_dct['label']):
    train_encoding = tokenizer(df_dct['content'][train].to_list(), padding =True)
    valid_encoding = tokenizer(df_dct['content'][test].to_list(), padding =True)

    train_dataset = DatasetSequenceVLSP(train_encoding,df_dct['label'][train])
    valid_dataset = DatasetSequenceVLSP(valid_encoding,df_dct['label'][test])
    
    train_datasets.append(train_dataset)
    valid_datasets.append(valid_dataset)

In [29]:
train_dataset.__len__(), valid_dataset.__len__()

(3205, 1602)

In [30]:
# !rm -r /kaggle/working/output/checkpoint-286

In [31]:
import evaluate

f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average="micro")


models = []
for i, (X, y) in enumerate(zip(train_datasets, valid_datasets)):
    phobert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base-v2", num_labels = 9, id2label = id2label, label2id = label2id).to("cuda")
    tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
#     if i == 1:
#         continue
    training_args = TrainingArguments(
        output_dir=f"/kaggle/working/output/{i}",
        learning_rate=2e-5,
        per_device_train_batch_size=64,
        per_device_eval_batch_size=64,
        num_train_epochs=6,
        evaluation_strategy="epoch",
        save_strategy="no",
#         save_total_limit = 2,
#         load_best_model_at_end=True,
    )


    trainer = Trainer(
        model=phobert,
        args=training_args,
        train_dataset=X,
        eval_dataset=y,
        compute_metrics=compute_metrics,
        optimizers = (optim.Adam(phobert.parameters(),lr=3e-5), None)
    )

    trainer.train()
    trainer.save_model(output_dir=f"/kaggle/working/output/{i}")
    
    models.append(trainer)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,F1
1,No log,0.961596,0.905802
2,No log,0.392494,0.942608
3,No log,0.246101,0.950094
4,No log,0.213400,0.950717
5,No log,0.239009,0.946974
6,No log,0.219838,0.949470
7,No log,0.187268,0.951341
8,No log,0.210748,0.950094
9,No log,0.195739,0.953837
10,0.342300,0.204096,0.954460


NameError: name 'models' is not defined

In [35]:
!zip -r /kaggle/working/models.zip /kaggle/working/output

  adding: kaggle/working/output/ (stored 0%)
  adding: kaggle/working/output/0/ (stored 0%)
  adding: kaggle/working/output/0/training_args.bin (deflated 48%)
  adding: kaggle/working/output/0/runs/ (stored 0%)
  adding: kaggle/working/output/0/runs/Oct27_06-52-17_da998a2ec56d/ (stored 0%)
  adding: kaggle/working/output/0/runs/Oct27_06-52-17_da998a2ec56d/events.out.tfevents.1698389538.da998a2ec56d.1999.0 (deflated 67%)
  adding: kaggle/working/output/0/pytorch_model.bin (deflated 7%)
  adding: kaggle/working/output/0/config.json (deflated 53%)


In [36]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
    
download_file('/kaggle/working/models.zip', 'out')

/kaggle/working/out.zip